In [52]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
from cleaning_utils import *
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
cwd = os.getcwd()
df = pd.read_csv(os.path.join(cwd, "MetObjects.txt"))

andrew_col_list = [i for i in range(2, df.shape[1], 3)]
andrew_col_list.insert(0, 0)
andrew_df = df.iloc[:, andrew_col_list]
andrew_df = andrew_df.astype(str, errors="ignore")



C:\Users\AMGra\AppData\Local\Temp\ipykernel_30464\500654137.py:2: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(cwd, "MetObjects.txt"))


In [54]:
artist_col_list = list(range(16, 28)) # making list of column index from Q->AB
artist_col_list.insert(0, 0) # Adding the first column as index
artist_df = df.iloc[: , artist_col_list] #subsets originas dataframe 'df'
artist_df = split_rows(artist_df)
artist_df.head()

,Object Number,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL
0,1979.486.1,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,nan,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459
1,1980.264.5,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",,1785,1844,nan,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648
2,67.265.9,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,67.265.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,67.265.11,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [55]:
andrew_artist_cols = [i for i in range(2, artist_df.shape[1], 3)]
andrew_artist_df = artist_df.iloc[:, andrew_artist_cols]

Andrew Cleaning

In [56]:
andrew_df.head(2)
andrew_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477804 entries, 0 to 477803
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Object Number      477804 non-null  object
 1   Is Timeline Work   477804 non-null  object
 2   Gallery Number     477804 non-null  object
 3   Object Name        477804 non-null  object
 4   Period             477804 non-null  object
 5   Portfolio          477804 non-null  object
 6   Artist Prefix      477804 non-null  object
 7   Artist Suffix      477804 non-null  object
 8   Artist Begin Date  477804 non-null  object
 9   Artist ULAN URL    477804 non-null  object
 10  Object Begin Date  477804 non-null  object
 11  Dimensions         477804 non-null  object
 12  City               477804 non-null  object
 13  Country            477804 non-null  object
 14  Locale             477804 non-null  object
 15  River              477804 non-null  object
 16  Link Resource      4

In [57]:
andrew_df['Is Timeline Work'] = andrew_df['Is Timeline Work'].astype(bool)

In [58]:
#Period
andrew_df["Period"] = andrew_df["Period"].replace(to_replace = '\(\?\)|\?', value = '', regex = True)
andrew_df["Period"] = andrew_df["Period"].replace(to_replace = '\(.*', value = '', regex = True)
andrew_df["Period"] = andrew_df["Period"].replace(to_replace = 'late|early|probably', value = '', regex = True)
andrew_df["Period"] = andrew_df["Period"].replace(to_replace = '\sor.*', value = '', regex = True)
andrew_df["Period"] = andrew_df["Period"].replace(to_replace = ',.*', value = '', regex = True)
andrew_df["Period"] = andrew_df["Period"].str.strip()
andrew_df[~andrew_df["Period"].str.contains("\(\?\)", regex=True)]["Period"].unique()

array(['nan', 'Edo period', 'Qing dynasty', 'Ming dynasty', 'Ming',
       'Yuan', 'Southern Song dynasty', 'Song dynasty', 'Yuan dynasty',
       'Goryeo dynasty', 'Southern Song', 'Dali kingdom', 'Meiji period',
       'Northern Song', 'Heian period', 'Muromachi', 'Kamakura period',
       'Shōwa period', 'Nanbokuchō period', 'Republic period',
       'Tang dynasty', 'Neolithic', 'Western Han dynasty', 'Liao dynasty',
       'Eastern Zhou dynasty', 'Neolithic period', 'Joseon dynasty',
       'Western Zhou dynasty', 'Unified Silla', 'Chola period',
       'Pala period', 'Early period', 'Bronze and Iron Age period',
       'Middle period', 'Central Javanese period',
       'Eastern Javanese period', 'Central', 'Singasari period',
       'Late period', 'Angkor period', 'Sukhothai period',
       'Thakuri-Malla periods', 'Polonnaruva period', 'post Gupta period',
       'Thakuri period', 'Late Neolithic period', 'Chiangsaen period',
       'Ratanakosin period', 'Gupta period', 'Eastern 

In [59]:
#Portfolio
andrew_df["Portfolio"] = andrew_df["Portfolio"].replace(to_replace = '\\r\\n', value = '', regex = True)
andrew_df = andrew_df.replace(r'^\s*$', np.nan, regex=True)

#andrew_df[~andrew_df["Portfolio"].str.contains("\(\?\)", regex=True)]["Portfolio"].unique()

In [60]:
#Object Begin Date
andrew_df['Object Begin Date']= pd.to_numeric(andrew_df['Object Begin Date'].copy())
andrew_df['Object Begin Date'].mask(andrew_df['Object Begin Date'] == 5000, -5000, inplace = True)


In [61]:
#Dimensions
andrew_df["Dimensions"] = andrew_df["Dimensions"].replace(to_replace = '\\r|\\n|\\t', value = '', regex = True)

In [62]:
#City
andrew_df["City"] = andrew_df["City"].replace(to_replace = '\(\?\)|\?', value = '', regex = True)
andrew_df["City"] = andrew_df["City"].replace(to_replace = '\(.*', value = '', regex = True)
andrew_df["City"] = andrew_df["City"].replace(to_replace = 'maybe|probably|possibly', value = '', regex = True)
andrew_df["City"] = andrew_df["City"].replace(to_replace = '\sor\s.*', value = '|', regex = True)
andrew_df["City"] = andrew_df["City"].replace(to_replace = '\sor.*', value = '|', regex = True)
# andrew_df["Period"] = andrew_df["Period"].replace(to_replace = ',.*', value = '', regex = True)
andrew_df["City"] = andrew_df["City"].str.strip()
andrew_df[andrew_df["City"].str.contains("\|", regex=True)]["City"].unique()
#andrew_df.groupby('City').nunique()

array(['Boston|Philadelphia', 'Boston|Charlestown', 'New York|Paris',
       'Hartford|New York', 'Barnstable|Bristol', 'Dordrecht|Amsterdam',
       'Mantua|New Geneva', 'East Cambridge|Sandwich',
       'Cincinnati|Trenton', 'Dedham|Medfield', 'Philadelphia|Middletown',
       'Newport|Middletown', 'Branford|Saybrook',
       'Philadelphia|Baltimore', 'New Lebanon|Watervliet', 'Kingston|',
       'Watervliet|New Lebanon', 'Boston|Salem', 'Guilford|Saybrook',
       'Boston|Newburyport', 'Philadelphia|Greensburg',
       'Chambersburg|Waynesboro', 'Cambridge|Boston',
       'Exeter|South Kingstown', 'New York|Albany',
       'Baltimore|Philadelphia', 'New York|Limoges', 'Atlanta|Limoges',
       'Fayetteville|Middletown', 'Wakefield|Gardner',
       'New York|White Mills', 'Coventry|Keene', 'Louisville|Zanesville',
       'Millville|Millville', 'Tarentum|Pittsburgh', 'Norfolk|Portsmouth',
       'Lanesborough|Springfield', 'Meriden|New York',
       'Manheim|Elizabeth', 'Philadelphia|

In [64]:
#Country
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '\(\?\)|\?', value = '', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '\(.*', value = '', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = 'maybe|probably|possibly', value = '', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '\sor\s.*', value = '|', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '\sor.*', value = '|', regex = True)
andrew_df["Country"] = andrew_df["Country"].str.replace(r'([\w\s]+)(\|(\1))+', r'\1')
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = ',\s?', value = '|', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '\|$', value = '', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = '(n|N)orth[a-z]+|(e|E)ast[a-z]+|(W|w)est[a-z]+|(S|s)outh[a-z]+', value = '', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = 'U\.S\.A\.|US|United Sates of America', value = 'United States', regex = True)
andrew_df["Country"] = andrew_df["Country"].replace(to_replace = 'UK', value = 'United Kingdom', regex = True)
andrew_df[~andrew_df["Country"].str.contains("^N", regex=True)]["Country"].unique()

array(['nan', 'Mexico', 'Guatemala', 'United States', 'England', 'Spain',
       'United States|England', 'United States|United States', 'France',
       'Canada', 'United States|France', 'England|France', 'China',
       'Italy', 'Japan', 'Germany', 'Czech Republic', 'England|Ireland',
       'Ireland|United States', 'Venezuela', 'England|Netherlands',
       'England|Belgium', 'France|Italy', 'Bermuda', 'Ireland', 'India',
       'England|United States',
       'United States|United States|United States',
       'France|NEW ZEALAND ROTORUA WHAKAREWAREWA',
       'China|NEW ZEALAND ROTORUA WHAKAREWAREWA', 'Germany|Italy',
       'Lisez et propagez nos Annales: "L\'Immacul‚e"',
       'France|Bernard Turner', 'France|Harold Copping',
       'Czech Republic|England', 'Belgium|France',
       'Belgium|England|France', 'United States|Germany',
       'France|Germany', 'Scotland', 'Germany|Czech Republic', 'Holland',
       'England|England', 'United States|Denmark',
       'Finland|Denmar

In [ ]:
andrew_artist_df = andrew_artist_df.replace(r'^\s*$', np.nan, regex=True)
andrew_artist_df.head(5)

In [ ]:
#Artist Prefix
andrew_artist_df["Artist Prefix"] = andrew_artist_df["Artist Prefix"].str.strip()
andrew_artist_df.groupby('Artist Prefix').nunique()

In [ ]:
#Artist Suffix
andrew_artist_df["Artist suffix"] = andrew_artist_df["Artist Prefix"].str.strip()
andrew_artist_df.groupby('Artist Suffix').nunique()

In [ ]:
#Artist Begin Date
andrew_artist_df["Artist Begin Date"] = andrew_artist_df["Artist Begin Date"].str.extract(r'(-*\d{4})')
andrew_artist_df.groupby('Artist Begin Date').nunique()
#andrew_df[~andrew_df["Artist Begin Date"].str.contains("\sor", regex=True)]["Artist Begin Date"].unique()

In [ ]:
city_col_list = [12] # making list of column index from Q->AB
city_col_list.insert(0, 0) # Adding the first column as index
city_df = andrew_df.iloc[: , city_col_list] #subsets originas dataframe 'df'
city_df = split_rows(city_df)

In [ ]:
city_df.groupby('City').nunique()

In [ ]:
country_col_list = [13] # making list of column index from Q->AB
country_col_list.insert(0, 0) # Adding the first column as index
country_df = andrew_df.iloc[: , country_col_list]
country_df = split_rows(country_df)
country_df.head()

In [ ]:
country_df.groupby('Country').nunique()